In [1]:
import warnings
import logging
warnings.filterwarnings("ignore")

In [2]:
import copy
import datetime


import numpy as np
import torch
from neuralforecast.core import NeuralForecast
from ray.rllib.algorithms.algorithm import Algorithm
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import Policy
from ray.tune.registry import register_env
from ray import tune
from tqdm.autonotebook import tqdm

from environments.environments import NeuralForecastingTradingEnv
from utils.utils import build_market_image, prep_forecasts, sharpe_reward

# warnings.filterwarnings('ignore')
# warnings.simplefilter('ignore')

2024-07-01 09:35:12,618	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-07-01 09:35:12,923	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:

target_pair='USD_JPY'
time_frame="1h"
symbol=target_pair.replace('_','')
symbol

'USDJPY'

In [4]:
model_path='FX_forecasting_model/'

model=NeuralForecast.load(model_path,verbose=False)

Seed set to 2
Seed set to 16
Seed set to 8
Seed set to 5


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
data=build_market_image(target_pair=symbol,time_frame='1h',axis=0,dir='data_forex')
data=data[data['symbol']==target_pair].copy()
data['ds']=data.index
data

,open,high,low,close,volume,y,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,...,feature_log_volume_15,feature_log_return_20,feature_log_volume_20,feature_log_return_25,feature_log_volume_25,feature_log_return_30,feature_log_volume_30,ds,unique_id,symbol
date_close,,,,,,,,,,,,,,,,,,,,,
2022-01-02 22:00:00,115.118,115.155,115.084,115.125,231,115.125,0.456522,0.5,-0.466667,-0.497260,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2022-01-02 22:00:00,20,USD_JPY
2022-01-02 23:00:00,115.110,115.158,115.099,115.144,848,115.144,0.500000,0.5,-0.466667,-0.497260,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2022-01-02 23:00:00,20,USD_JPY
2022-01-03 00:00:00,115.142,115.294,115.124,115.280,1771,115.280,-0.500000,-0.5,-0.433333,-0.494521,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2022-01-03 00:00:00,20,USD_JPY
2022-01-03 01:00:00,115.279,115.280,115.182,115.213,1401,115.213,-0.456522,-0.5,-0.433333,-0.494521,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2022-01-03 01:00:00,20,USD_JPY
2022-01-03 02:00:00,115.214,115.279,115.207,115.278,946,115.278,-0.413043,-0.5,-0.433333,-0.494521,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2022-01-03 02:00:00,20,USD_JPY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-17 14:00:00,157.888,157.962,157.832,157.856,6254,157.856,0.108696,-0.5,0.033333,-0.039726,...,0.935094,0.003656,0.559256,0.003668,-0.456785,0.002302,-0.822412,2024-06-17 14:00:00,20,USD_JPY
2024-06-17 15:00:00,157.857,157.894,157.784,157.811,4218,157.811,0.152174,-0.5,0.033333,-0.039726,...,-0.645565,0.003205,0.630656,0.002728,-1.093065,0.004528,-1.144863,2024-06-17 15:00:00,20,USD_JPY
2024-06-17 16:00:00,157.812,157.818,157.729,157.784,3513,157.784,0.195652,-0.5,0.033333,-0.039726,...,-0.285545,0.002379,0.478982,0.003155,-0.667194,0.005236,-1.427697,2024-06-17 16:00:00,20,USD_JPY


In [7]:
split_date=datetime.datetime(year= 2024, month= 6, day=1)
end_date=split_date+datetime.timedelta(days=14)


hf_data=data.copy()

hf_train_data=hf_data.groupby('symbol').apply(lambda x: x[:split_date])
hf_test_data=hf_data.groupby('symbol').apply(lambda x: x[split_date:])

hf_train_data=hf_train_data.reset_index(level=0,drop=True).reset_index()
hf_test_data=hf_test_data.reset_index(level=0,drop=True).reset_index()


trade_data=data[data['symbol']==target_pair].copy()
train_data=trade_data[:split_date]
test_data=trade_data[split_date:end_date]
# hf_test_data

In [8]:
train_data.head(5)

,open,high,low,close,volume,y,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,...,feature_log_volume_15,feature_log_return_20,feature_log_volume_20,feature_log_return_25,feature_log_volume_25,feature_log_return_30,feature_log_volume_30,ds,unique_id,symbol
date_close,,,,,,,,,,,,,,,,,,,,,
2022-01-02 22:00:00,115.118,115.155,115.084,115.125,231,115.125,0.456522,0.5,-0.466667,-0.497260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-01-02 22:00:00,20,USD_JPY
2022-01-02 23:00:00,115.110,115.158,115.099,115.144,848,115.144,0.500000,0.5,-0.466667,-0.497260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-01-02 23:00:00,20,USD_JPY
2022-01-03 00:00:00,115.142,115.294,115.124,115.280,1771,115.280,-0.500000,-0.5,-0.433333,-0.494521,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-01-03 00:00:00,20,USD_JPY
2022-01-03 01:00:00,115.279,115.280,115.182,115.213,1401,115.213,-0.456522,-0.5,-0.433333,-0.494521,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-01-03 01:00:00,20,USD_JPY
2022-01-03 02:00:00,115.214,115.279,115.207,115.278,946,115.278,-0.413043,-0.5,-0.433333,-0.494521,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-01-03 02:00:00,20,USD_JPY


In [9]:
train_forecast_array,train_df=prep_forecasts(model=model,df=train_data)
test_forecast_array,test_df=prep_forecasts(df=test_data,model=model)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [10]:
train_env_config=dict(
                name=f'{symbol}_train',
                forecast_horizon=7,
                context_length=35,
                # initial_position=0,
                reward_function=sharpe_reward,
                forecasts=train_forecast_array,
                df = train_df, # Your dataset with your custom features
                positions = [ -.5,-.25,.25, .5], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
                trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
                borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
                max_episode_duration=168,
                verbose=0
                )

test_env_config=copy.deepcopy(train_env_config)
test_env_config['name']=f'{symbol}_test'
test_env_config['df'] = test_df
test_env_config['forecasts']=test_forecast_array



In [11]:
train_env=NeuralForecastingTradingEnv(**train_env_config)

In [12]:
obs=train_env.reset()
terminated = truncated = False
# env.action_space.sa
while not terminated and not truncated:
    action = train_env.action_space.sample()
    obs, reward, terminated, truncated, info = train_env.step(action)
info['reward']
# train_env.results_metrics
# train_env.historical_info.columns

-0.026552553229189813

In [13]:


register_env("train_env", lambda config: NeuralForecastingTradingEnv(**train_env_config))
register_env("test_env", lambda config: NeuralForecastingTradingEnv(**test_env_config))


In [14]:

def trial_str_creator(trial):
    return "{}_{}_trading_agent".format(trial.trainable_name, trial.trial_id)



In [23]:
expiriment_path='/Users/sebastiancoombs/git/MultiTrader/Agent_FX/PPO_2024-06-30_11-49-05'
trainer=tune.ExperimentAnalysis(expiriment_path,default_mode='max',default_metric='env_runners/episode_reward_mean')


In [24]:
df_results=trainer.results_df
custom_cols=[
                'config/lr',
                'config/gamma',
                'config/lambda',
                'config/kl_coeff',
                'config/model/fcnet_hiddens',

                ]
custom_cols+=[c for c in df_results.filter(like='episode_reward').columns]
custom_cols+=[c for c in df_results.filter(like='custom_metrics').columns]
custom_cols=[c for c in custom_cols if 'sampler_results' not in c ]
custom_cols=[c for c in custom_cols if 'hist_stats' not in c ]
custom_cols=list(set(custom_cols))
df_results=df_results.sort_values('env_runners/episode_reward_mean',ascending=False)
df_results[custom_cols]

,config/keep_per_episode_custom_metrics,env_runners/custom_metrics/position_changes_max,env_runners/custom_metrics/portfolio_value_min,config/kl_coeff,env_runners/episode_reward_mean,config/lr,config/model/fcnet_hiddens,env_runners/episode_reward_min,config/lambda,env_runners/episode_reward_max,env_runners/custom_metrics/portfolio_value_mean,env_runners/custom_metrics/portfolio_value_max,env_runners/custom_metrics/position_changes_min,config/gamma,env_runners/custom_metrics/position_changes_mean
trial_id,,,,,,,,,,,,,,,
ab3af569,False,71.0,1006.543483,0.037562,81.012402,6.074078e-06,"[1024, 1024]",24.761863,0.955820,115.336049,1045.973548,1096.160639,25.0,0.900365,41.67
9fc8b3f3,False,74.0,1009.665417,0.061098,65.852149,7.749627e-06,"[512, 512]",-2.029658,0.949826,103.503430,1038.597047,1097.277296,4.0,0.955536,47.83
61926199,False,90.0,1007.836869,0.146206,65.675585,5.687335e-06,"[512, 512]",-0.247703,0.996872,109.664034,1038.942366,1093.315705,24.0,0.840481,46.18
9ecebea5,False,51.0,979.903356,0.003336,8.032991,1.679901e-06,"[256, 256]",-31.595195,0.999055,57.101896,1001.109760,1014.924381,11.0,0.883162,30.52
c584a142,False,60.0,979.723704,0.015720,7.346607,4.637492e-07,"[1024, 1024]",-39.799575,0.999940,45.460048,1002.842728,1017.942981,20.0,0.856063,35.11
4d9b9b1e,False,95.0,975.317271,0.104614,6.598742,1.913576e-06,"[256, 256]",-35.573960,0.959553,42.765950,1001.257264,1014.712586,41.0,0.962360,65.50
0d8721c9,False,24.0,986.111221,0.185777,3.480031,1.545478e-06,"[2048, 2048]",-36.134300,0.913235,46.946094,1000.133586,1011.684279,2.0,0.928735,10.51
9bdfc646,False,7.0,975.328541,0.005938,2.761412,2.215331e-06,"[512, 512]",-27.336131,0.923321,46.964746,1000.245204,1016.668400,0.0,0.979566,2.27
36a4b16e,False,36.0,971.074877,0.098265,2.549626,1.853007e-06,"[512, 512]",-35.863607,0.928609,47.595714,1000.746095,1022.858563,0.0,0.879498,1.39


In [53]:
# trainer.results_df.filter(like='point')

,config/checkpoint_trainable_policies_only
trial_id,
4c50425d,False
211ad79b,False
71656610,False
9fc8b3f3,False
c65747b4,False
ab3af569,False
6abbcd99,False
4ccad93a,False
1f7827f7,False


In [25]:
trainer.best_checkpoint

Checkpoint(filesystem=local, path=/Users/sebastiancoombs/git/MultiTrader/Agent_FX/PPO_2024-06-30_11-49-05/PPO_ab3af569_trading_agent/checkpoint_000000)

In [17]:
## this come from the fitt notebook
best_path=trainer.best_checkpoint.path
policy_path=best_path+'/policies/default_policy'

In [32]:
policy1=Algorithm.from_checkpoint(trainer.best_checkpoint.path)

In [54]:
pickle_cp_dir=f'FX_Agent/{target_pair}_production_checkpoint'
prod_cp_dir=f'FX_Agent/{target_pair}_production_agent'
prod_onnx_dir=f'FX_Agent/{target_pair}_production_agent_onnx'

In [55]:
# num_epochs=200
# bar=tqdm(range(num_epochs))
# for i in bar:
#     algo.train()
#     if i % 100 == 0:
#         checkpoint_dir = algo.save(pickle_cp_dir).checkpoint.path
#         bar.set_description(f"Epoch {i}: Checkpoint saved in directory {checkpoint_dir}")

In [56]:
import onnx

In [57]:

# Tell RLlib to store an individual policy checkpoint (only for "pol1") inside
# /tmp/my_policy_checkpoint
policy1.export_checkpoint(prod_cp_dir)
policy1.export_model(prod_onnx_dir,onnx=18)


In [37]:
input_dict={
        "obs": torch.from_numpy(np.array([[0.0, 0.1, 0.2]], dtype=np.float32)),
    }

In [38]:
onnx_model = onnx.load(prod_onnx_dir+'/model.onnx')
onnx.checker.check_model(onnx_model)

In [39]:

# Use the `from_checkpoint` utility of the Policy class:
agent = Policy.from_checkpoint(prod_cp_dir)

In [40]:
test_env=NeuralForecastingTradingEnv(**test_env_config)

In [41]:
import onnxruntime as ort

In [42]:
ort_sess = ort.InferenceSession(prod_onnx_dir+'/model.onnx')


In [43]:
input1=ort_sess.get_inputs()[0]
input1.shape


['batch_size', 53]

In [44]:
obs

array([ 1.26758286e+02,  5.26915503e+00,  1.27754494e+02,  1.26787209e+02,
        1.26810997e+02,  1.26759247e+02,  5.33982563e+00,  1.27718819e+02,
        1.26802048e+02,  1.26880997e+02,  1.26731644e+02,  5.33185720e+00,
        1.27825729e+02,  1.26796371e+02,  1.26858002e+02,  1.26708130e+02,
        5.31853199e+00,  1.27708641e+02,  1.26781563e+02,  1.26843002e+02,
        1.26707970e+02,  5.35796356e+00,  1.27761444e+02,  1.26797432e+02,
        1.26814003e+02,  1.26689499e+02,  5.36774874e+00,  1.27822212e+02,
        1.26844009e+02,  1.26737999e+02,  2.39130437e-01, -3.33333343e-01,
        2.66666681e-01, -1.08219177e-01,  1.27357452e+02,  1.27576859e+02,
        1.28429977e+02,  1.61061145e-03, -6.79136455e-01, -3.71659826e-03,
       -4.49220598e-01, -4.14877944e-03, -7.25526929e-01, -7.99872261e-03,
        5.62695712e-02, -8.96866992e-03,  1.13526237e+00, -8.53074528e-03,
        3.87828916e-01, -6.73011411e-03,  4.42739904e-01, -5.00000000e-01,
       -5.01152694e-01], 

In [45]:
obs,info=test_env.reset()

input_dict={'obs': np.array([obs]),'state_ins':np.zeros(1,dtype=np.float32)}
input_dict

{'obs': array([[ 1.5604419e+02,  5.3671575e+00,  1.5340097e+02,  1.5608664e+02,
          1.5614600e+02,  1.5602487e+02,  5.4273882e+00,  1.5335083e+02,
          1.5613255e+02,  1.5609599e+02,  1.5603468e+02,  5.4269810e+00,
          1.5350108e+02,  1.5608830e+02,  1.5574600e+02,  1.5605670e+02,
          5.4019246e+00,  1.5333653e+02,  1.5612383e+02,  1.5631500e+02,
          1.5604985e+02,  5.4480329e+00,  1.5341074e+02,  1.5614125e+02,
          1.5621201e+02,  1.5606482e+02,  5.4511447e+00,  1.5349614e+02,
          1.5618855e+02,  1.5612399e+02, -6.5217391e-02, -1.6666667e-01,
         -3.6666667e-01, -7.2602741e-02,  1.5533275e+02,  1.5573566e+02,
          1.5671257e+02, -1.1398640e-03, -4.5886794e-01,  2.7204268e-03,
         -3.3127129e-01,  5.4096207e-03, -7.1114039e-01,  8.3386358e-03,
         -2.3151136e-03,  7.6281750e-03, -1.3966705e+00,  5.8993432e-03,
         -9.7018170e-01, -1.4726128e-03,  3.7041819e-01,  5.0000000e-01,
          5.0000000e-01]], dtype=float32),
 

In [46]:

outputs = ort_sess.run(None,input_dict)
outputs
state=outputs[-1]
action_proba=outputs[0]
action=int(np.argmax(action_proba))
action

4

In [ ]:
eval_profit=[]
eval_market=[]
for i in tqdm( range(100)):
    obs,info=test_env.reset()
    states=np.ones(1,dtype=np.float32)
    action=None
    reward=0
    terminated = truncated = False

    while not terminated and not truncated:

        input_dict={'obs': np.array([obs]),'state_ins':states}
        outputs = ort_sess.run(None,input_dict)

        state=outputs[-1]
        action_proba=outputs[0]
        action=int(np.argmax(action_proba))
        

        # action = train_env.action_space.sample()
        # print(action)
        next_obs, reward, terminated, truncated, info = test_env.step(action)

        obs=next_obs
    metrics=test_env.get_metrics()
    eval_market.append(float(metrics['Market Return'].replace('%','')))
    eval_profit.append(float(metrics['Portfolio Return'].replace('%','')))

    # info['portfolio_valuation']

In [ ]:
import matplotlib.pyplot as plt
plt.plot(eval_profit,label='agent_return')
plt.plot(eval_market,label='market_return')
plt.legend()
np.mean(eval_profit),np.mean(eval_market)

In [ ]:
eval_profit=[]
eval_market=[]
for i in tqdm( range(100)):
    obs,info=test_env.reset()
    action=None
    reward=0
    terminated = truncated = False

    while not terminated and not truncated:

        action,_,states = agent.compute_single_action(obs,explore=False)
        # action = train_env.action_space.sample()
        # print(action)
        next_obs, reward, terminated, truncated, info = test_env.step(action)

        obs=next_obs
    metrics=test_env.get_metrics()
    eval_market.append(float(metrics['Market Return'].replace('%','')))
    eval_profit.append(float(metrics['Portfolio Return'].replace('%','')))

    # info['portfolio_valuation']


In [ ]:

plt.plot(eval_profit,label='agent_return')
plt.plot(eval_market,label='market_return')
plt.legend()
np.mean(eval_profit),np.mean(eval_market)

In [ ]:
obs,info=test_env.reset()
action=None
reward=0
terminated = truncated = False


while not terminated and not truncated:

    action ,_,states= agent.compute_single_action(obs,explore=False)
    # action = train_env.action_space.sample()
    # print(action)
    next_obs, reward, terminated, truncated, info = test_env.step(action)

    obs=next_obs
info['reward'],info[ 'portfolio_valuation'],test_env.get_metrics()


